In [35]:
import snap 
import csv
import pandas as pd
import random
import networkx as nx
from node2vec import Node2Vec
#from deepwalk import DeepWalk
#from graphembedding.sdne import SDNE
#from gem.embedding.hope import HOPE

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import ParameterGrid
from scipy.sparse import coo_matrix


%run Fairness.ipynb
%run Embeddings_Classification.ipynb

https://snap.stanford.edu/data/github-social.html 

Read the file with the edges and the files with the target/attribute 

In [2]:
path_edges = 'Datasets/git_web_ml/musae_git_edges.csv'
path_target = 'Datasets/git_web_ml/musae_git_target.csv'

create graph based of the edge file

In [3]:

df_graph = pd.read_csv(path_edges)
G = nx.from_pandas_edgelist(df_graph, 'id_1', 'id_2')

In [4]:
print(G.number_of_nodes())
print(G.number_of_edges())

37700
289003


create Dataframe based on the csv file

In [5]:
df = pd.read_csv(path_target)
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [6]:
df.head()

,id,name,ml_target
0,0,Eiryyy,0
1,1,shawflying,0
2,2,JpMCarrilho,1
3,3,SuhwanCha,0
4,4,sunilangadi2,1


Peform random sample on the list of nodes to get a graph with 1000 files

In [7]:
list_nodes = []
for nodes in G.nodes():
    list_nodes.append(nodes)
random.seed(42)

# Perform random sampling
sampled_nodes = random.sample(list_nodes, 1000)
print(len(sampled_nodes))

    
    

1000


In [8]:
G_nx = nx.Graph()
for node in sampled_nodes:
        G_nx.add_node(node)
print(G_nx.number_of_nodes())

1000


Add edges to the new sampled Graph 

In [9]:
for node in G_nx.nodes():
    node_x_neighbors = set() 
    for neighbor in G.neighbors(node):
        node_x_neighbors.add(neighbor)
    for nd in node_x_neighbors:
        if node != nd and G_nx.has_node(nd):
                G_nx.add_edge(node, nd)


In [10]:
print(G_nx.number_of_edges())


131


In [11]:
print(nx.is_connected(G_nx))

False


Find the largest connected components and ceate a new graph based on that

In [12]:
largest_cc = max(nx.connected_components(G_nx), key=len)
G_new = G_nx.subgraph(largest_cc)

In [13]:
print(nx.is_connected(G_new))

True


In [25]:
print(G_new.number_of_nodes())
print(G_new.number_of_edges())

32
33


In [26]:
list_nodes = [nodes for nodes in G_new.nodes()]
print(len(list_nodes))

32


In [27]:
df_new= df.loc[list_nodes]
df_new = df_new.reset_index(drop=True)

In [28]:
df_target = df_new['ml_target']
df_name = df_new['name']

In [30]:
df_target.head(10)

0    0
1    0
2    0
3    0
4    0
5    1
6    0
7    0
8    0
9    1
Name: ml_target, dtype: int64

In [31]:
# Resetting the index of the nodes from 0 to number of nodes for SINE and AE
new_indices = range(0, G_new.number_of_nodes())
mapping = {old_index: new_index for new_index, old_index in zip(new_indices, G_new.nodes())}
G_new = nx.relabel_nodes(G_new, mapping)

Add attributes to the graph

In [32]:
'add synthetic gender '
list_gender = {i: i % 2 for i in range(G_new.number_of_nodes())}
print(len(list_gender))
nx.set_node_attributes(G_new, list_gender, 'gender')
nx.set_node_attributes(G_new, df_name, 'name')

32


In [33]:
print(len(list_gender.values()))
datF = pd.DataFrame(list_gender.values())

32


In [36]:
df_features = datF.apply(pd.to_numeric, errors='coerce')
row_indices, col_indices = np.where(~np.isnan(datF.values))


# Create the COO sparse matrix
coo_matrix = coo_matrix((datF.values.ravel(), (row_indices, col_indices)), shape=(datF.shape[0], datF.shape[1]))

In [37]:
# Parameters
parameter_grid = {
    'dimensions': [16, 32],
    'walk_length': [20, 30],
    'walk_number': [25, 75],
    'workers': [2, 4]
}

#apply SINE embedding, find the best embedding with the grid search algorithm
best_params, best_fpr_w,best_fpr_m, dataframe = grid_search_SINE(x_Graph, coo_matrix,parameter_grid,  list(df_target), list_gender)

done create embeding vectors
[0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[10  0]
 [ 0  0]]
done iteration
[[10  0]
 [ 0  0]]
PPV_f and PPV_m is equal: True
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[10  0]
 [ 0  0]]
[[10  0]
 [ 0  0]]
[[10  0]
 [ 0  0]]
fpF: 0.0
fpM: 0


C:\Users\nagel\AppData\Local\Temp\ipykernel_20828\1038381496.py:8: RuntimeWarning: invalid value encountered in longlong_scalars
  fnr_f = fn_f / (tp_f + fn_f)
C:\Users\nagel\AppData\Local\Temp\ipykernel_20828\398579461.py:10: RuntimeWarning: invalid value encountered in longlong_scalars
  ppv_f = tp_f/(tp_f + fn_f)


done create embeding vectors
[0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[10  0]
 [ 0  0]]
done iteration
[[10  0]
 [ 0  0]]
PPV_f and PPV_m is equal: True
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[10  0]
 [ 0  0]]
[[10  0]
 [ 0  0]]
[[10  0]
 [ 0  0]]
fpF: 0.0
fpM: 0


C:\Users\nagel\AppData\Local\Temp\ipykernel_20828\1038381496.py:8: RuntimeWarning: invalid value encountered in longlong_scalars
  fnr_f = fn_f / (tp_f + fn_f)
C:\Users\nagel\AppData\Local\Temp\ipykernel_20828\398579461.py:10: RuntimeWarning: invalid value encountered in longlong_scalars
  ppv_f = tp_f/(tp_f + fn_f)


done create embeding vectors
[0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[10  0]
 [ 0  0]]
done iteration
[[10  0]
 [ 0  0]]
PPV_f and PPV_m is equal: True
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[10  0]
 [ 0  0]]
[[10  0]
 [ 0  0]]
[[10  0]
 [ 0  0]]
fpF: 0.0
fpM: 0


...

C:\Users\nagel\AppData\Local\Temp\ipykernel_20828\1038381496.py:8: RuntimeWarning: invalid value encountered in longlong_scalars
  fnr_f = fn_f / (tp_f + fn_f)
C:\Users\nagel\AppData\Local\Temp\ipykernel_20828\398579461.py:10: RuntimeWarning: invalid value encountered in longlong_scalars
  ppv_f = tp_f/(tp_f + fn_f)


done create embeding vectors
[0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[10  0]
 [ 0  0]]
done iteration
[[10  0]
 [ 0  0]]
PPV_f and PPV_m is equal: True
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[10  0]
 [ 0  0]]
[[10  0]
 [ 0  0]]
[[10  0]
 [ 0  0]]
fpF: 0.0
fpM: 0


C:\Users\nagel\AppData\Local\Temp\ipykernel_20828\1038381496.py:8: RuntimeWarning: invalid value encountered in longlong_scalars
  fnr_f = fn_f / (tp_f + fn_f)
C:\Users\nagel\AppData\Local\Temp\ipykernel_20828\398579461.py:10: RuntimeWarning: invalid value encountered in longlong_scalars
  ppv_f = tp_f/(tp_f + fn_f)


done create embeding vectors
[0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[10  0]
 [ 0  0]]
done iteration
[[10  0]
 [ 0  0]]
PPV_f and PPV_m is equal: True
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[10  0]
 [ 0  0]]
[[10  0]
 [ 0  0]]
[[10  0]
 [ 0  0]]
fpF: 0.0
fpM: 0


C:\Users\nagel\AppData\Local\Temp\ipykernel_20828\1038381496.py:8: RuntimeWarning: invalid value encountered in longlong_scalars
  fnr_f = fn_f / (tp_f + fn_f)
C:\Users\nagel\AppData\Local\Temp\ipykernel_20828\398579461.py:10: RuntimeWarning: invalid value encountered in longlong_scalars
  ppv_f = tp_f/(tp_f + fn_f)


In [38]:
print(best_params, best_fpr_w,best_fpr_m, dataframe)

{'dimensions': 16, 'walk_length': 20, 'walk_number': 25, 'workers': 2} 0.0 0                            Female  Male
Group Fairness                0.0   0.0
Predictive Parity             0.0   0.0
False Positive Error Rate     0.0   0.0
Equalized Opportunity         NaN   0.0
Equalized Odds PPV            NaN   0.0
Equalized Odds FPR            0.0   0.0


In [39]:
# Beispielparameter-Raster
parameter_grid = {
    'dimensions': [16, 32],
    'walk_length': [20, 30],
    'walk_number': [25, 75],
    'workers': [2, 4]
}

#apply AE embedding, find the best embedding with the grid search algorithm
best_params, best_fpr_w,best_fpr_m, dataframe = grid_search_AE(x_Graph, coo_matrix,parameter_grid,  list(df_target), list_gender)

done create embeding vectors
[0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[10  0]
 [ 0  0]]
done iteration
[[10  0]
 [ 0  0]]
PPV_f and PPV_m is equal: True
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[10  0]
 [ 0  0]]
[[10  0]
 [ 0  0]]
[[10  0]
 [ 0  0]]
fpF: 0.0
fpM: 0


C:\Users\nagel\AppData\Local\Temp\ipykernel_20828\1038381496.py:8: RuntimeWarning: invalid value encountered in longlong_scalars
  fnr_f = fn_f / (tp_f + fn_f)
C:\Users\nagel\AppData\Local\Temp\ipykernel_20828\398579461.py:10: RuntimeWarning: invalid value encountered in longlong_scalars
  ppv_f = tp_f/(tp_f + fn_f)


done create embeding vectors
[0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[10  0]
 [ 0  0]]
done iteration
[[10  0]
 [ 0  0]]
PPV_f and PPV_m is equal: True
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[10  0]
 [ 0  0]]
[[10  0]
 [ 0  0]]
[[10  0]
 [ 0  0]]
fpF: 0.0
fpM: 0


C:\Users\nagel\AppData\Local\Temp\ipykernel_20828\1038381496.py:8: RuntimeWarning: invalid value encountered in longlong_scalars
  fnr_f = fn_f / (tp_f + fn_f)
C:\Users\nagel\AppData\Local\Temp\ipykernel_20828\398579461.py:10: RuntimeWarning: invalid value encountered in longlong_scalars
  ppv_f = tp_f/(tp_f + fn_f)


done create embeding vectors
[0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[10  0]
 [ 0  0]]
done iteration
[[10  0]
 [ 0  0]]
PPV_f and PPV_m is equal: True
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[10  0]
 [ 0  0]]
[[10  0]
 [ 0  0]]
[[10  0]
 [ 0  0]]
fpF: 0.0
fpM: 0


...

C:\Users\nagel\AppData\Local\Temp\ipykernel_20828\1038381496.py:8: RuntimeWarning: invalid value encountered in longlong_scalars
  fnr_f = fn_f / (tp_f + fn_f)
C:\Users\nagel\AppData\Local\Temp\ipykernel_20828\398579461.py:10: RuntimeWarning: invalid value encountered in longlong_scalars
  ppv_f = tp_f/(tp_f + fn_f)


done create embeding vectors
[0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[10  0]
 [ 0  0]]
done iteration
[[10  0]
 [ 0  0]]
PPV_f and PPV_m is equal: True
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[10  0]
 [ 0  0]]
[[10  0]
 [ 0  0]]
[[10  0]
 [ 0  0]]
fpF: 0.0
fpM: 0


C:\Users\nagel\AppData\Local\Temp\ipykernel_20828\1038381496.py:8: RuntimeWarning: invalid value encountered in longlong_scalars
  fnr_f = fn_f / (tp_f + fn_f)
C:\Users\nagel\AppData\Local\Temp\ipykernel_20828\398579461.py:10: RuntimeWarning: invalid value encountered in longlong_scalars
  ppv_f = tp_f/(tp_f + fn_f)


done create embeding vectors
[0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[10  0]
 [ 0  0]]
done iteration
[[10  0]
 [ 0  0]]
PPV_f and PPV_m is equal: True
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[10  0]
 [ 0  0]]
[[10  0]
 [ 0  0]]
[[10  0]
 [ 0  0]]
fpF: 0.0
fpM: 0


C:\Users\nagel\AppData\Local\Temp\ipykernel_20828\1038381496.py:8: RuntimeWarning: invalid value encountered in longlong_scalars
  fnr_f = fn_f / (tp_f + fn_f)
C:\Users\nagel\AppData\Local\Temp\ipykernel_20828\398579461.py:10: RuntimeWarning: invalid value encountered in longlong_scalars
  ppv_f = tp_f/(tp_f + fn_f)


In [40]:
print(best_params, best_fpr_w,best_fpr_m, dataframe)

{'dimensions': 16, 'walk_length': 20, 'walk_number': 25, 'workers': 2} 0.0 0                            Female  Male
Group Fairness                0.0   0.0
Predictive Parity             0.0   0.0
False Positive Error Rate     0.0   0.0
Equalized Opportunity         NaN   0.0
Equalized Odds PPV            NaN   0.0
Equalized Odds FPR            0.0   0.0


In [41]:
# Parameter
parameter_grid = {
    'dimensions': [16, 32],
    'walk_length': [20, 30],
    'num_walks': [25, 75],
    'workers': [2, 4]
}

#apply Node2Vec embedding, find the best embedding with the grid search algorithm
best_params, best_fpr_w,best_fpr_m, dataframe = perform_grid_search(G_new, parameter_grid, list(df_target), list(df_name))


Computing transition probabilities:   0%|          | 0/32 [00:00<?, ?it/s]

done create embeding vectors
[0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[9 0]
 [1 0]]
done iteration
[[9 0]
 [1 0]]
PPV_f and PPV_m is equal: True
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[9 0]
 [1 0]]
[[9 0]
 [1 0]]
[[9 0]
 [1 0]]
fpF: 0.0
fpM: 0


Computing transition probabilities:   0%|          | 0/32 [00:00<?, ?it/s]

done create embeding vectors
[0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[9 0]
 [1 0]]
done iteration
[[9 0]
 [1 0]]
PPV_f and PPV_m is equal: True
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[9 0]
 [1 0]]
[[9 0]
 [1 0]]
[[9 0]
 [1 0]]
fpF: 0.0
fpM: 0


Computing transition probabilities:   0%|          | 0/32 [00:00<?, ?it/s]

...

done create embeding vectors
[0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[10  0]
 [ 0  0]]
done iteration
[[10  0]
 [ 0  0]]
PPV_f and PPV_m is equal: True
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[10  0]
 [ 0  0]]
[[10  0]
 [ 0  0]]
[[10  0]
 [ 0  0]]
fpF: 0.0
fpM: 0


C:\Users\nagel\AppData\Local\Temp\ipykernel_20828\1038381496.py:8: RuntimeWarning: invalid value encountered in longlong_scalars
  fnr_f = fn_f / (tp_f + fn_f)
C:\Users\nagel\AppData\Local\Temp\ipykernel_20828\398579461.py:10: RuntimeWarning: invalid value encountered in longlong_scalars
  ppv_f = tp_f/(tp_f + fn_f)


Computing transition probabilities:   0%|          | 0/32 [00:00<?, ?it/s]

done create embeding vectors
[0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[10  0]
 [ 0  0]]
done iteration
[[10  0]
 [ 0  0]]
PPV_f and PPV_m is equal: True
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[10  0]
 [ 0  0]]
[[10  0]
 [ 0  0]]
[[10  0]
 [ 0  0]]
fpF: 0.0
fpM: 0


C:\Users\nagel\AppData\Local\Temp\ipykernel_20828\1038381496.py:8: RuntimeWarning: invalid value encountered in longlong_scalars
  fnr_f = fn_f / (tp_f + fn_f)
C:\Users\nagel\AppData\Local\Temp\ipykernel_20828\398579461.py:10: RuntimeWarning: invalid value encountered in longlong_scalars
  ppv_f = tp_f/(tp_f + fn_f)


In [42]:
print(f"Best Parameters: {best_params}")
print(f"Best Fpr_w: {best_fpr_w}")
print(f"Best Fpr_m: {best_fpr_m}")

Best Parameters: {'dimensions': 16, 'num_walks': 25, 'walk_length': 20, 'workers': 2}
Best Fpr_w: 0.0
Best Fpr_m: 0


In [43]:
print(dataframe)

                           Female  Male
Group Fairness                0.0   0.0
Predictive Parity             0.0   0.0
False Positive Error Rate     0.0   0.0
Equalized Opportunity         1.0   0.0
Equalized Odds PPV            0.0   0.0
Equalized Odds FPR            0.0   0.0


In [44]:
# Parameter
parameter_grid = {
    'window': [16, 32],
    'walk_length': [20, 30],
    'number_of_random_walks': [25, 75],
}
#apply DeepWalk embedding, find the best embedding with the grid search algorithm
best_params, best_fpr_f,best_fpr_m, dataframe =  perform_grid_search_DeepWalk(G_new, parameter_grid, list(df_target), list(df_name))

25
20
16


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:00<00:00, 761.75it/s]


[0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[9 0]
 [1 0]]
fpF: 0.0
fpM: 0
[[9 0]
 [1 0]]
PPV_f and PPV_m is equal: True
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[9 0]
 [1 0]]
[[9 0]
 [1 0]]
[[9 0]
 [1 0]]
25
20
32


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:00<00:00, 941.05it/s]


[0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[9 0]
 [1 0]]
fpF: 0.0
fpM: 0
[[9 0]
 [1 0]]
PPV_f and PPV_m is equal: True
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[9 0]
 [1 0]]
[[9 0]
 [1 0]]
[[9 0]
 [1 0]]
25
30
16


...

[0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[10  0]
 [ 0  0]]
fpF: 0.0
fpM: 0
[[10  0]
 [ 0  0]]
PPV_f and PPV_m is equal: True
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[10  0]
 [ 0  0]]
[[10  0]
 [ 0  0]]
[[10  0]
 [ 0  0]]
75
30
16


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:00<00:00, 189.08it/s]


[0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[9 0]
 [1 0]]
fpF: 0.0
fpM: 0
[[9 0]
 [1 0]]
PPV_f and PPV_m is equal: True
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[9 0]
 [1 0]]
[[9 0]
 [1 0]]
[[9 0]
 [1 0]]
75
30
32


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:00<00:00, 146.68it/s]


[0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[9 0]
 [1 0]]
fpF: 0.0
fpM: 0
[[9 0]
 [1 0]]
PPV_f and PPV_m is equal: True
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[9 0]
 [1 0]]
[[9 0]
 [1 0]]
[[9 0]
 [1 0]]


In [45]:
print(f"Best Parameters: {best_params}")
print(f"Best Fpr: {best_fpr_f}")
print(f"Best Fpr: {best_fpr_m}")
print(dataframe)

Best Parameters: {'number_of_random_walks': 25, 'walk_length': 20, 'window': 16}
Best Fpr: 0.0
Best Fpr: 0
                           Female  Male
Group Fairness                0.0   0.0
Predictive Parity             0.0   0.0
False Positive Error Rate     0.0   0.0
Equalized Opportunity         1.0   0.0
Equalized Odds PPV            0.0   0.0
Equalized Odds FPR            0.0   0.0
